In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 50)
import numpy as np
import matplotlib.pyplot as plt
import precision_recall_cutoff as prc
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-448-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'turnover.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## 1. Using the pandas library to read the csv data file and create a data-frame called turnover

turnover = pd.read_csv(file_content_stream)

turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [2]:
## 2. Changing sales and salary from labels to dummy variables

## Sales
turnover = pd.concat([turnover.drop(columns = ['sales']), pd.get_dummies(turnover['sales'])], axis = 1)

## Salary
turnover = pd.concat([turnover.drop(columns = ['salary']), pd.get_dummies(turnover['salary'])], axis = 1)

turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,IT,RandD,accounting,hr,management,marketing,product_mng,sales,support,technical,high,low,medium
0,0.38,0.53,2,157,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [3]:
## 3. Engineering the interactions/features from In-Class assignment 9 (the ones from the decision tree)

turnover['Interaction_1'] = np.where((turnover['satisfaction_level'] <= 0.465) & (turnover['number_project'] <= 2.5) & 
                                     (turnover['last_evaluation'] <= 0.575), 1, 0)

turnover['Interaction_2'] = np.where((turnover['satisfaction_level'] <= 0.465) & (turnover['number_project'] > 2.5) & 
                                     (turnover['satisfaction_level'] <= 0.115), 1, 0)

turnover['Interaction_3'] = np.where((turnover['satisfaction_level'] > 0.465) & (turnover['time_spend_company'] <= 4.5) & 
                                     (turnover['average_montly_hours'] <= 290.5), 1, 0)

In [4]:
## 4. Using satisfaction level, last evaluation, number project, average montly hours, time spend company, Work accident, 
## promotion last 5years, sales (dummy variables), and salary (dummy variables) and interactions/features (from part 3) 
## as the input variables and left as the target variable to split the data into two data-frames 

## Defining the input and target variables
X = turnover.drop(columns = ['left'])
Y = turnover['left']

## Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [6]:
## 5. Using train data-frame and the top 5 features to perform a hyper-tuning job on the Ada Boost model using the 
## GridSearchCV function 

## Defining input variables
X_train = X_train[['Interaction_3', 'Interaction_1', 'satisfaction_level', 'time_spend_company', 'number_project']]
X_test = X_test[['Interaction_3', 'Interaction_1', 'satisfaction_level', 'time_spend_company', 'number_project']]

## Defining the parameter dictionary
Ada_param_grid = {'n_estimators': [100, 300, 500], 'base_estimator__min_samples_split': [10, 15], 
                  'base_estimator__min_samples_leaf': [5, 7], 'base_estimator__max_depth': [3, 5, 7], 
                  'learning_rate': [0.001, 0.01, 0.1]}

## Running GridSearchCV with 3 folds
ada_grid_search = GridSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), Ada_param_grid, cv = 3, 
                               scoring = 'f1', n_jobs = -1).fit(X_train, Y_train)

## Extracting the best hyper-parameters
ada_grid_search.best_params_

{'base_estimator__max_depth': 7,
 'base_estimator__min_samples_leaf': 5,
 'base_estimator__min_samples_split': 10,
 'learning_rate': 0.01,
 'n_estimators': 500}

In [7]:
## Building an AdaBoost model with the best hyper-parameter combination from grid search
ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(min_samples_split = 10, min_samples_leaf = 5, max_depth = 7), 
                            n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)

## Predicting on the test data-frame
ada_preds = ada_md.predict_proba(X_test)[:, 1]

## Extracting the predicted labels
ada_labels = prc.precision_recall_cutoff(Y_test, ada_preds)

## Computing the classification report
print(classification_report(Y_test, ada_labels))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      2286
           1       0.95      0.94      0.94       714

    accuracy                           0.97      3000
   macro avg       0.97      0.96      0.96      3000
weighted avg       0.97      0.97      0.97      3000



In [9]:
## 6. Using train data-frame and the top 5 features to perform a hyper-tuning job on the Ada Boost model using the RandomizedSearchCV 
## function and the hyper-parameter dictionary from part 5

## Running RandomizedSearchCV with 3 folds
ada_random_search = RandomizedSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), Ada_param_grid, cv = 3, 
                               scoring = 'f1', n_jobs = -1,  n_iter = 50).fit(X_train, Y_train)

## Extracting the best hyper-parameters
ada_random_search.best_params_

{'n_estimators': 500,
 'learning_rate': 0.01,
 'base_estimator__min_samples_split': 10,
 'base_estimator__min_samples_leaf': 5,
 'base_estimator__max_depth': 7}

In [10]:
## Building an AdaBoost model with the best hyper-parameter combination from random search
ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(min_samples_split = 10, min_samples_leaf = 5, max_depth = 7), 
                            n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)

## Predicting on the test data-frame
ada_preds = ada_md.predict_proba(X_test)[:, 1]

## Extracting the predicted labels
ada_labels = prc.precision_recall_cutoff(Y_test, ada_preds)

## Computing the classification report
print(classification_report(Y_test, ada_labels))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      2286
           1       0.95      0.94      0.94       714

    accuracy                           0.97      3000
   macro avg       0.97      0.96      0.96      3000
weighted avg       0.97      0.97      0.97      3000



In [ ]:
## 7. Using the results from part 5 and 6, we would use RandomizedSearchCV because it produced the exact same result as SearchGridCV
## but in a fraction of the time.